In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from PIL import Image

# Constants
IMAGE_SIZE = 64
LATENT_DIM = 100
EPOCHS = 100
SAVE_INTERVAL = 1000
IMG_ROWS = IMAGE_SIZE
IMG_COLS = IMAGE_SIZE
IMG_CHANNELS = 3  # RGB images

# Load UTKFace dataset
def load_utkface(data_path, img_size):
    images = []
    conditions = []

    for file in os.listdir(data_path):
        if file.endswith(".jpg"):
            parts = file.split("_")
            if len(parts) < 2:
                continue

            try:
                age = int(parts[0])
                gender = int(parts[1])

                img_path = os.path.join(data_path, file)
                img = Image.open(img_path)
                img = img.resize((img_size, img_size))
                img_array = np.array(img)

                if img_array.shape == (img_size, img_size, 3):
                    images.append(img_array)
                    conditions.append([age, gender])
            except Exception as e:
                print(f"Error processing file {file}: {e}")

    images = np.array(images, dtype="float32")
    conditions = np.array(conditions, dtype="float32")

    images = (images / 127.5) - 1.0  # Normalize to [-1, 1]
    conditions[:, 0] /= 100  # Normalize age to [0, 1]

    return images, conditions

# One-hot encode gender
def preprocess_data(data_path):
    images, conditions = load_utkface(data_path, IMAGE_SIZE)
    ages = conditions[:, 0]
    genders = to_categorical(conditions[:, 1], num_classes=2)
    conditions_normalized = np.hstack((ages.reshape(-1, 1), genders))
    return images, conditions_normalized

# Hyperparameters grid
param_grid = {
    'learning_rate': [0.0001, 0.0002, 0.0005],
    'batch_size': [32, 64, 128],
}

# Results storage
results = []

# Function to build the generator model
def build_generator(latent_dim):
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(2,))
    x = Concatenate()([noise_input, label_input])
    x = Dense(128, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(np.prod((64, 64, 3)), activation='tanh')(x)
    img_output = Reshape((64, 64, 3))(x)
    model = Model([noise_input, label_input], img_output)
    return model

# Function to build the discriminator model
def build_discriminator():
    img = Input(shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS))
    label = Input(shape=(2,))
    label_embedding = Dense(np.prod((IMG_ROWS, IMG_COLS, IMG_CHANNELS)))(label)
    label_embedding = Reshape((IMG_ROWS, IMG_COLS, IMG_CHANNELS))(label_embedding)
    concatenated = Concatenate(axis=-1)([img, label_embedding])
    x = Flatten()(concatenated)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    validity = Dense(1, activation='sigmoid')(x)
    model = Model([img, label], validity)
    return model

# Function to build the cGAN model
def build_cgan(generator, discriminator):
    discriminator.trainable = False
    noise = Input(shape=(LATENT_DIM,))
    label = Input(shape=(2,))
    img = generator([noise, label])
    validity = discriminator([img, label])
    model = Model([noise, label], validity)
    return model

# Train the cGAN model
def train(generator, discriminator, cgan, epochs, batch_size, save_interval, images, conditions):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, images.shape[0], half_batch)
        imgs = images[idx]
        label_real = conditions[idx, :2]
        noise = np.random.normal(0, 1, (half_batch, LATENT_DIM))
        fake_labels = np.random.randint(0, 2, (half_batch, 2))
        gen_imgs = generator.predict([noise, fake_labels])
        real_preds = discriminator.predict([imgs, label_real])
        fake_preds = discriminator.predict([gen_imgs, fake_labels])
        d_loss_real = tf.keras.losses.BinaryCrossentropy(from_logits=True)(np.ones_like(real_preds), real_preds)
        d_loss_fake = tf.keras.losses.BinaryCrossentropy(from_logits=True)(np.zeros_like(fake_preds), fake_preds)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        fake_labels = np.random.randint(0, 2, (batch_size, 2))
        valid_labels = np.ones((batch_size, 1))
        g_loss = cgan.train_on_batch([noise, fake_labels], valid_labels)

        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss}] [G loss: {g_loss}]")
    return d_loss, g_loss

# Function to perform grid search
def grid_search(param_grid, images, conditions, epochs):
    for learning_rate in param_grid['learning_rate']:
        for batch_size in param_grid['batch_size']:
            # Compile models with current hyperparameters
            optimizer = Adam(learning_rate, 0.5)
            generator = build_generator(LATENT_DIM)
            discriminator = build_discriminator()
            cgan = build_cgan(generator, discriminator)
            discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
            cgan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
            print(f"Training with learning rate: {learning_rate}, batch size: {batch_size}")
            d_loss, g_loss = train(generator, discriminator, cgan, epochs, batch_size, SAVE_INTERVAL, images, conditions)
            results.append({'learning_rate': learning_rate, 'batch_size': batch_size, 'd_loss': d_loss, 'g_loss': g_loss})
    
    # Sort results by generator loss (lower is better)
    results_sorted = sorted(results, key=lambda x: x['g_loss'])
    
    # Plot results
    for result in results_sorted:
        print(f"LR: {result['learning_rate']}, BS: {result['batch_size']} | D Loss: {result['d_loss']} | G Loss: {result['g_loss']}")
    
    return results_sorted

# Main execution
data_path = r"C:\Users\Marinand\OneDrive - Carrier Corporation\Documents\Grad School\Advanced Machine Learning\Week 4\Assignment 4\UTKFace"
images, conditions = preprocess_data(data_path)
results_sorted = grid_search(param_grid, images, conditions, EPOCHS)

c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1']. Received: the structure of inputs=('*', '*')
  warnings.warn(


Training with learning rate: 0.0001, batch size: 32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_9', 'keras_tensor_10']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
0 [D loss: 0.7218005657196045] [G loss: [array(0.6914531, dtype=float32), array(0.59375, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━

c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_26', 'keras_tensor_27']. Received: the structure of inputs=('*', '*')
  warnings.warn(


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
0 [D loss: 0.7293820977210999] [G loss: [array(0.72040176, dtype=float32), array(0., dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_208', 'keras_tensor_209']. Received: the structure of inputs=('*', '*')
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step 


c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_217', 'keras_tensor_218']. Received: the structure of inputs=('*', '*')
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step 
0 [D loss: 0.7326220273971558] [G loss: [array(0.69892955, dtype=float32), array(0.2265625, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━